In [ ]:
import cv2
import numpy as np

# Pfad zum Video
video_path = "./data/results/corona/2024_0205_094407_006_3sec.mp4"

# OpenCV Video-Capture-Objekt erstellen
cap = cv2.VideoCapture(video_path)

# Farbe für die Visualisierung des optischen Flusses
farbe = (255, 0, 0)

# Schleife über jedes Frame im Video
while(cap.isOpened()):
    ret, frame = cap.read()
    if not ret:
        break
    
    # Konvertierung des Frames in Graustufen
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Berechnung des optischen Flusses
    if 'prev_gray' in locals():
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        
        # Visualisierung des optischen Flusses auf dem Frame
        for y in range(0, frame.shape[0], 10):
            for x in range(0, frame.shape[1], 10):
                dx, dy = flow[y, x]
                cv2.line(frame, (x, y), (int(x+dx), int(y+dy)), farbe, 2)
    
    # Aktualisierung des vorherigen Graustufenframes
    prev_gray = gray
    
    # Anzeige des Frames mit optischem Fluss
    cv2.imshow('Optischer Fluss', frame)
    
    # Überprüfung auf Tastendruck zum Beenden der Schleife
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Freigabe des Video-Capture-Objekts und Schließen aller Fenster
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np

# Pfad zum Video
video_path = "./data/results/corona/2024_0205_094407_006_3sec.mp4"

# OpenCV Video-Capture-Objekt erstellen
cap = cv2.VideoCapture(video_path)

# Liste zum Speichern der akkumulierten optischen Flussvektoren
accumulated_flow = None

# Anzahl der Frames für die Mittelung des optischen Flusses
num_frames_to_average = 5

# Schleife über jedes Frame im Video
while(cap.isOpened()):
    ret, frame = cap.read()
    if not ret:
        break
    
    # Konvertierung des Frames in Graustufen
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Berechnung des optischen Flusses
    if 'prev_gray' in locals():
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        
        # Akkumulation der optischen Flussvektoren
        if accumulated_flow is None:
            accumulated_flow = flow
        else:
            accumulated_flow += flow
        
        # Wenn die Anzahl der Frames erreicht ist, Mittelwert berechnen und als Heatmap darstellen
        if cap.get(cv2.CAP_PROP_POS_FRAMES) % num_frames_to_average == 0:
            average_flow = accumulated_flow / num_frames_to_average
            
            # Heatmap des durchschnittlichen optischen Flusses erstellen
            magnitude, angle = cv2.cartToPolar(average_flow[..., 0], average_flow[..., 1])
            hsv = np.zeros_like(frame)
            hsv[..., 0] = angle * 180 / np.pi / 2
            hsv[..., 1] = 255
            hsv[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
            heatmap = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
            
            # Anzeige der Heatmap
            cv2.imshow('Average Optical Flow Heatmap', heatmap)
            
            # Zurücksetzen der akkumulierten Flusswerte für den nächsten Durchlauf
            accumulated_flow = None
    
    # Aktualisierung des vorherigen Graustufenframes
    prev_gray = gray
    
    # Überprüfung auf Tastendruck zum Beenden der Schleife
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Freigabe des Video-Capture-Objekts und Schließen aller Fenster
cap.release()
cv2.destroyAllWindows()


In [2]:
import cv2
import numpy as np

# Pfad zum Video
video_path = "./data/results/corona/2024_0205_094407_006_3sec.mp4"

# OpenCV Video-Capture-Objekt erstellen
cap = cv2.VideoCapture(video_path)

# Farbe für die Visualisierung des optischen Flusses
farbe = (0, 255, 0)

# Schleife über jedes Frame im Video
while(cap.isOpened()):
    ret, frame = cap.read()
    if not ret:
        break
    
    # Konvertierung des Frames in Graustufen
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Berechnung des optischen Flusses
    if 'prev_gray' in locals():
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        
        # Erstellen eines leeren Bildes für die Richtung des optischen Flusses
        direction_image = np.zeros_like(frame)
        
        # Zeichnen von Richtungspfeilen auf dem Bild
        for y in range(0, frame.shape[0], 20):
            for x in range(0, frame.shape[1], 20):
                dx, dy = flow[y, x]
                cv2.arrowedLine(direction_image, (x, y), (int(x+dx), int(y+dy)), farbe, 1)
        
        # Anzeigen des Frames mit optischem Fluss
        cv2.imshow('Optischer Fluss', direction_image)
    
    # Aktualisierung des vorherigen Graustufenframes
    prev_gray = gray
    
    # Überprüfung auf Tastendruck zum Beenden der Schleife
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Freigabe des Video-Capture-Objekts und Schließen aller Fenster
cap.release()
cv2.destroyAllWindows()


In [4]:
import cv2
import numpy as np

# Pfad zum Video
video_path = "./data/results/corona/2024_0205_094407_006_3sec.mp4"

# OpenCV Video-Capture-Objekt erstellen
cap = cv2.VideoCapture(video_path)

# Farbe für die Visualisierung des optischen Flusses
farbe = (0, 255, 0)

# Unteres und oberes Limit für die Vektorlänge
min_length = 2
max_length = 4

# Schleife über jedes Frame im Video
while(cap.isOpened()):
    ret, frame = cap.read()
    if not ret:
        break
    
    # Konvertierung des Frames in Graustufen
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Berechnung des optischen Flusses
    if 'prev_gray' in locals():
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        
        # Erstellen eines leeren Bildes für die Richtung des optischen Flusses
        direction_image = np.zeros_like(frame)
        
        # Zeichnen von Richtungspfeilen auf dem Bild, nur wenn der Fluss im Bereich zwischen min_length und max_length liegt
        for y in range(0, frame.shape[0], 15):
            for x in range(0, frame.shape[1], 15):
                dx, dy = flow[y, x]
                length = np.sqrt(dx**2 + dy**2)
                if min_length <= length <= max_length:
                    cv2.arrowedLine(direction_image, (x, y), (int(x+dx*2), int(y+dy*2)), farbe, 1)
        
        # Anzeigen des Frames mit optischem Fluss
        cv2.imshow('Optischer Fluss', direction_image)
    
    # Aktualisierung des vorherigen Graustufenframes
    prev_gray = gray
    
    # Überprüfung auf Tastendruck zum Beenden der Schleife
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Freigabe des Video-Capture-Objekts und Schließen aller Fenster
cap.release()
cv2.destroyAllWindows()
